In [ ]:
import numpy
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Input
from keras.models import Model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import sys
import matplotlib.dates as mdates
import datetime as dt

In [ ]:
def showGraph(data):
    plt.figure()
    plt.plot(data)
    plt.show()
def saveGraph(data, file_name):
    plt.figure(dpi=360)
    plt.plot(data)
    plt.savefig('results/' + file_name)
# convert dataset to x and y
# def create_dataset(dataset, look_back=1):
#     dataX, dataY = [], []
#     for i in range(len(dataset) - look_back - 1):
#         a = dataset[i:(i+look_back), 0]
#         dataX.append(a)
#         dataY.append(dataset[i + look_back, 0])
#     return numpy.array(dataX), numpy.array(dataY)
def relative_error(xs, ys):
    error = 0
    zero = 0
    for i in range(len(xs)):
        if ys[i] != 0:
            error += abs(xs[i]-ys[i])*100 / ys[i]
        else:
            zero += 1
    error /= (len(xs) - zero)
    return error
def read_data(file_name):
    return pandas.read_csv('datas/' + file_name, usecols=[1], sep='|')
def saveGraphWithDate(dates, y_axis, file_name, format_date):
    plt.figure(dpi=360)
    x = [dt.datetime.strptime(d,format_date).date() for d in dates]
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%Y'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=6))
    plt.plot(x,y_axis)
    plt.gcf().autofmt_xdate()
    plt.savefig('results/' + file_name)
def create_dataset(dataset, look_back=1):
    trainX, trainY, testX, testY = [], [], [], []
    for i in range(len(dataset) - look_back):
        x = dataset[i:(i+look_back), :]
        if i <= int(len(dataset) * 0.67):
            trainX.append(x)
            trainY.append(dataset[i+look_back])
        else:
            testX.append(x)
            testY.append(dataset[i+look_back])
    return numpy.array(trainX), numpy.array(trainY), numpy.array(testX), numpy.array(testY)

In [ ]:
datafile = 'data_stock_market.csv'
result_path = 'raw'
format_date='%d/%m/%Y'
epochs = 10
look_back = 1

print('2. LSTM ')
# dataset = read_data(datafile)
# saveGraph(dataset, result_path + '/data')
dataframe = pandas.read_csv('datas/' + datafile, sep='|')
saveGraphWithDate(dataframe['date'], dataframe['close_price'], result_path + '/data', format_date)
dataset = dataframe['close_price'].values.reshape(dataframe['close_price'].shape[0], 1)

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
# dataset.loc[-1] = [max(dataset.values)*2]
dataset = scaler.fit_transform(dataset)
# dataset = dataset[:-1]

# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
# reshape into X=t and Y=t+1

trainX, trainY, testX, testY = create_dataset(dataset, look_back)
trainX = trainX.reshape(trainX.shape[0], look_back, 1)
testX = testX.reshape(testX.shape[0], look_back, 1)
trainY = trainY.reshape(trainY.shape[0], 1)
testY = testY.reshape(testY.shape[0], 1)

In [ ]:
model_input = Input(shape=(look_back, 1))
model_output = LSTM(1, return_sequences=False)(model_input)
model = Model(input=model_input, output=model_output)

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=epochs, batch_size=1, verbose=2)

# loop predict only the next day and fit to model
print('----- Predict Trend -----')
predict = []
dates_test = dataframe['date'].values[(len(dataset) - len(testY) - 1):(len(dataset)-1)]

for index, today_close_price in enumerate(testX):
    predict_tomorrow = model.predict(numpy.array([today_close_price]))[0]
    predict.append(predict_tomorrow)
    model.fit(numpy.array([today_close_price]), numpy.array([testY[index]]), epochs=epochs, batch_size=1, verbose=0)

In [ ]:
predict = scaler.inverse_transform(predict)
# testX = scaler.inverse_transform(testX[:, 0])
testY = scaler.inverse_transform(testY)

In [ ]:
with open('results/' + result_path + '/result.csv', 'w+') as file:
    file.write('close_price|predict\n')
    for i in range(len(predict)):
        file.write(str(testY[i][0]) + '|' + str(predict[i][0]) + '\n')
plt.figure(dpi=200)
plt.plot(testY, linewidth=0.5)
plt.plot(predict, linewidth=0.5)
plt.show()

In [ ]:
today = scaler.inverse_transform(testX[:, look_back - 1])

In [ ]:
_trend_reallity = [int(numpy.sign(b[0] - a[0])) for a, b in zip(today, testY)]
_trend_predict = [int(numpy.sign(b[0] - a[0])) for a, b in zip(today, predict)]
_correct = [1 if a == b else 0 for a, b in zip(_trend_reallity, _trend_predict)]
print('Percent correct: %.2f' % (100*sum(_correct)/len(_correct)))

In [ ]:
for i in range(1, 5):
    print(i)

In [ ]:
list(range(1, 10)) + list(range(12, 40, 2)) + list(range(45, 100, 5)) + list(range(120, 240, 20))

In [1]:
import numpy
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Input
from keras.models import Model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import sys
import matplotlib.dates as mdates
import datetime as dt

def saveGraph(data, file_name):
    plt.figure(dpi=360)
    plt.plot(data)
    plt.savefig('results/' + file_name)
def relative_error(xs, ys):
    error = 0
    zero = 0
    for i in range(len(xs)):
        if ys[i] != 0:
            error += abs(xs[i]-ys[i])*100 / ys[i]
        else:
            zero += 1
    error /= (len(xs) - zero)
    return error
def read_data(file_name):
    return pandas.read_csv('datas/' + file_name, usecols=[1], sep='|')
def saveGraphWithDate(dates, y_axis, file_name, format_date):
    plt.figure(dpi=360)
    x = [dt.datetime.strptime(d,format_date).date() for d in dates]
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%Y'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=6))
    plt.plot(x,y_axis)
    plt.gcf().autofmt_xdate()
    plt.savefig('results/' + file_name)
def create_dataset(dataset, look_back=1):
    trainX, trainY, testX, testY = [], [], [], []
    for i in range(len(dataset) - look_back):
        x = dataset[i:(i+look_back), :]
        if i <= int(len(dataset) * 0.67):
            trainX.append(x)
            trainY.append(dataset[i+look_back])
        else:
            testX.append(x)
            testY.append(dataset[i+look_back])
    return numpy.array(trainX), numpy.array(trainY), numpy.array(testX), numpy.array(testY)



datafile = 'data_stock_market.csv'
result_path = 'raw'
format_date='%d/%m/%Y'
epochs = 1

percents_correct = []
look_backs = list(range(1, 3))

for look_back in look_backs:
    dataframe = pandas.read_csv('datas/' + datafile, sep='|')
    dataset = dataframe['close_price'].values.reshape(dataframe['close_price'].shape[0], 1)

    # normalize the dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)

    # split into train and test sets
    train_size = int(len(dataset) * 0.67)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

    trainX, trainY, testX, testY = create_dataset(dataset, look_back)
    trainX = trainX.reshape(trainX.shape[0], look_back, 1)
    testX = testX.reshape(testX.shape[0], look_back, 1)
    trainY = trainY.reshape(trainY.shape[0], 1)
    testY = testY.reshape(testY.shape[0], 1)

    model_input = Input(shape=(look_back, 1))
    model_output = LSTM(1, return_sequences=False)(model_input)
    model = Model(input=model_input, output=model_output)

    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(trainX, trainY, epochs=epochs, batch_size=1, verbose=2)


    print('----- Predict Trend -----')
    predict = []
    dates_test = dataframe['date'].values[(len(dataset) - len(testY) - 1):(len(dataset)-1)]

    for index, today_close_price in enumerate(testX):
        predict_tomorrow = model.predict(numpy.array([today_close_price]))[0]
        predict.append(predict_tomorrow)
        model.fit(numpy.array([today_close_price]), numpy.array([testY[index]]), epochs=epochs, batch_size=1, verbose=0)

    predict = scaler.inverse_transform(predict)
    testY = scaler.inverse_transform(testY)

    with open('results/' + result_path + '/result.csv', 'w+') as file:
        file.write('close_price|predict\n')
        for i in range(len(predict)):
            file.write(str(testY[i][0]) + '|' + str(predict[i][0]) + '\n')

    today = scaler.inverse_transform(testX[:, look_back - 1])

    _trend_reallity = [int(numpy.sign(b[0] - a[0])) for a, b in zip(today, testY)]
    _trend_predict = [int(numpy.sign(b[0] - a[0])) for a, b in zip(today, predict)]
    _correct = [1 if a == b else 0 for a, b in zip(_trend_reallity, _trend_predict)]
    _percent_correct = 100*sum(_correct)/len(_correct)
    percents_correct.append({'look_back': look_back, 'percent': _percent_correct})

with open('results/' + result_path + '/percents.csv', 'w+') as file:
    file.write('look_back|percent\n')
    for i in percents_correct:
        file.write(str(i['look_back']) + '|' + str(i['percent']) + '\n')


Using TensorFlow backend.
d:\python35\lib\site-packages\ipykernel_launcher.py:85: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ls...)`


Epoch 1/1
2s - loss: 0.0814
----- Predict Trend -----
Epoch 1/1
3s - loss: 0.0737
----- Predict Trend -----
